In [1]:
import torch
import copy

In [2]:
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)


In [3]:
# Helper function to evaluate model
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x)
            loss = criterion(preds, y)
            total_loss += loss.item() * x.size(0)
            correct += (preds.argmax(dim=1) == y).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [4]:
# Function to prune weights by percentage
def prune_weights(model, mask, prune_percentage):
    new_mask = []
    with torch.no_grad():
        for param, m in zip(model.parameters(), mask):
            tensor = param.data.cpu().abs() * m
            # Only prune if there are non-zero values in the tensor
            non_zero_tensor = tensor[tensor > 0]
            if non_zero_tensor.numel() > 0:
                threshold = torch.quantile(non_zero_tensor, prune_percentage)
                new_m = torch.where(tensor <= threshold, torch.zeros_like(m), m)
            else:
                new_m = m  # If no non-zero weights, keep the original mask
            param.data.mul_(new_m.to(param.device))
            new_mask.append(new_m)
    return new_mask


# Create mask for initial weights
def init_mask(model):
    return [torch.ones_like(param, device='cpu') for param in model.parameters()]

In [5]:
# Lottery Ticket Hypothesis implementation
def lottery_ticket(model, train_loader, val_loader, optimizer, criterion,
                   device='cuda', epochs=2, prune_percentage=0.2, rounds=10, final_epochs=50):
    model = model.to(device)

    # Save original weights
    original_weights = copy.deepcopy(model.state_dict())
    best_acc = 0
    best_mask = None

    mask = init_mask(model)

    for round in range(rounds):
        print(f'\n--- Round {round+1} ---')

        # Reset weights to original initialization with mask applied
        model.load_state_dict(original_weights)
        with torch.no_grad():
            for param, m in zip(model.parameters(), mask):
                param.data.mul_(m.to(device))

        # Retrain
        for epoch in range(epochs):
            train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
            val_loss, val_acc = evaluate(model, val_loader, criterion, device)
            print(f'Epoch {epoch+1}/{epochs} - Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        if val_acc > best_acc:
            best_acc = val_acc
            best_mask = copy.deepcopy(mask)

        # Prune weights
        mask = prune_weights(model, mask, prune_percentage)

    print(f'\nBest validation accuracy: {best_acc:.4f}')

    # Final training with best mask
    print('\n--- Final training with best mask ---')
    model.load_state_dict(original_weights)
    with torch.no_grad():
        for param, m in zip(model.parameters(), best_mask):
            param.data.mul_(m.to(device))

    for epoch in range(final_epochs):
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        print(f'Final Epoch {epoch+1}/{final_epochs} - Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

# Example Usage (assuming all components provided):
# lottery_ticket(model, train_loader, val_loader, optimizer, criterion, device='cuda')


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [7]:
np.random.seed(42)  # Set the seed for NumPy

# Set the seed for PyTorch (CPU and GPU)
torch.manual_seed(42)

# If you're using CUDA (GPU), set the seed for CUDA as well
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)  # For all GPUs (if you have more than one)

In [8]:
# 1. Download the dataset (OpenML: id 180)
dataset = openml.datasets.get_dataset(180)
X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format='dataframe')

In [9]:
# 2. Preprocessing: Numeric features only, standardize, encode labels (classes are 1-7)
X_numeric = X.select_dtypes(include=[np.number])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric.values.astype(np.float32))

le = LabelEncoder()
y_encoded = le.fit_transform(y)  # Converts to 0...6
y_encoded = y_encoded.astype(np.int64)

In [10]:
# 3. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [11]:
# 4. PyTorch Dataset
class CovertypeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = CovertypeDataset(X_train, y_train)
test_ds = CovertypeDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=256, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=256)

In [12]:
from dpn_3.dpn import DPN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DPN(X_train.shape[1], 192 + len(le.classes_), len(le.classes_), False).to(device)

In [13]:
# 6. Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
lottery_ticket(model, train_loader, test_loader, optimizer, criterion)


--- Round 1 ---
Epoch 1/2 - Train loss: 0.9540, Val loss: 0.8344, Val Acc: 0.6849
Epoch 2/2 - Train loss: 0.8376, Val loss: 0.8101, Val Acc: 0.6932

--- Round 2 ---
Epoch 1/2 - Train loss: 0.9518, Val loss: 0.8337, Val Acc: 0.6848
Epoch 2/2 - Train loss: 0.8384, Val loss: 0.8109, Val Acc: 0.6953

--- Round 3 ---
Epoch 1/2 - Train loss: 0.9371, Val loss: 0.8277, Val Acc: 0.6884
Epoch 2/2 - Train loss: 0.8325, Val loss: 0.8049, Val Acc: 0.6991

--- Round 4 ---
Epoch 1/2 - Train loss: 0.9320, Val loss: 0.8278, Val Acc: 0.6869
Epoch 2/2 - Train loss: 0.8318, Val loss: 0.8052, Val Acc: 0.6970

--- Round 5 ---
Epoch 1/2 - Train loss: 0.9322, Val loss: 0.8257, Val Acc: 0.6887
Epoch 2/2 - Train loss: 0.8304, Val loss: 0.8042, Val Acc: 0.6971

--- Round 6 ---
Epoch 1/2 - Train loss: 0.9385, Val loss: 0.8279, Val Acc: 0.6876
Epoch 2/2 - Train loss: 0.8323, Val loss: 0.8070, Val Acc: 0.6959

--- Round 7 ---
Epoch 1/2 - Train loss: 0.9493, Val loss: 0.8288, Val Acc: 0.6870
Epoch 2/2 - Train loss: